# Compte Rendu : Projet TC4 - BIARD David & BERRIEN Samuel

# Modèle complètement naïf (TP 1)

In [1]:
import numpy as np
import random
import pickle

# Compte le nombre d'occurence de chaque mot
def mkCountWord(data):
    vocab = {}
    for phrase in data:
        for mot in phrase:
            if mot[0] in vocab:
                vocab[mot[0]] += 1
            else:
                vocab[mot[0]] = 1
    return vocab

# Comptre le nombre d'occurence de chaque tags et retourn la liste de tous les tags
def allTags(data):
    tags = {}
    for phrase in data:
        for mot in phrase:
            if mot[1] in tags:
                tags[mot[1]] += 1
            else:
                tags[mot[1]] = 1
    l_tags = np.array([t for t in tags.keys()])
    return tags, l_tags

# Compte le nombre de pairs (x, y) dans les données de train
def pair_count(data_train):
    res = {}
    for phrase in data_train:
        for t_u in phrase:
            if t_u in res:
                res[t_u] += 1
            else:
                res[t_u] = 1
    return res

# Calcul P(Y = y | X = x)
def p_y_sachant_x(count_of_pairs, count_words, y, x):
    if (x, y) in count_of_pairs:
        return round(count_of_pairs[(x, y)] / count_words[x], 3)
    else:
        return 0
    
# Calcul la probabilité ci-dessus pour tous les tags de la listes de tags
def vect_prob_for_all_y(count_of_pairs, count_words, l_tags, x):
    res = [(y, p_y_sachant_x(count_of_pairs, count_words, y, x)) for y in l_tags]
    return res

# Tri les prédictions pour obtenir la plus élevée
def prediction(vect_prob):
    res = sorted(vect_prob,key=lambda x: x[1], reverse=True)[0]
    return res[0]

# Evaluation des performances du modèle
def eval_model(data_test, data_train, count_of_pairs, count_words, l_tags):
    nb_error = 0
    for phrases in data_test:
        for t_u in phrases:
            x = t_u[0]
            out = prediction(vect_prob_for_all_y(count_of_pairs, count_words, l_tags, x))
            if(out != t_u[1]):
                nb_error += 1
    accuracy = ((1 - nb_error / (len(np.array(data_test).sum())))*100)
    return round(accuracy,1)

# Transforme les données à l'aider des fonctions décrites ci-dessus
def process_data(data_train, n):
    count = mkCountWord(data_train)
    list_winf10 = [k for k, v in count.items() if v > n]
    new_data_train = [[t_u for t_u in l if t_u[0] in list_winf10] for l in data_train]
    return new_data_train

Calcul des paramètres servant aux prédiction du modèle.

In [2]:
def compute_params(data):
    tags, l_tags = allTags(data)
    count_words = mkCountWord(data)
    count_of_pairs = pair_count(data)
    return tags, l_tags, count_words, count_of_pairs

# Modèle HMM (first & second Order) TP2 &TP3

### Description de la classe HMM

In [3]:
import nltk
import numpy as np
import sys
import pickle

# Some words in test could be unseen during training, or out of the vocabulary (OOV) even during the training. 
# To manage OOVs, all words out the vocabulary are mapped on a special token: UNK defined as follows: 
UNK = "<unk>" 
UNKid = 0 

class HMM:
        def __init__(self, state_list, observation_list,
                 transition_proba_2 = None,
                 transition_proba_3 = None,
                 observation_proba = None,
                 initial_state_proba = None):
            """Builds a new Hidden Markov Model
            state_list is the list of state symbols [q_0...q_(N-1)]
            observation_list is the list of observation symbols [v_0...v_(M-1)]
            transition_proba is the transition probability matrix
                [a_ij] a_ij = Pr(Y_(t+1)=q_i|Y_t=q_j)
            observation_proba is the observation probablility matrix
                [b_ki] b_ki = Pr(X_t=v_k|Y_t=q_i)
            initial_state_proba is the initial state distribution
                [pi_i] pi_i = Pr(Y_0=q_i)"""
            print("HMM created with: ")
            self.N = len(state_list) # The number of states
            self.M = len(observation_list) # The number of words in the vocabulary
            print(str(self.N)+" states")
            print(str(self.M)+" observations")
            self.omega_Y = state_list # Keep the vocabulary of tags
            self.omega_X = observation_list # Keep the vocabulary of words
            # Init. of the 3 distributions : observation, transition and initial states
            if transition_proba_3 is None:
                self.transition_proba_3 = np.zeros( (self.N, self.N, self.N), float) 
            else:
                self.transition_proba_3=transition_proba_3
            if transition_proba_2 is None:
                self.transition_proba_2 = np.zeros( (self.N, self.N), float) 
            else:
                self.transition_proba_2=transition_proba_2
            if observation_proba is None:
                self.observation_proba = np.zeros( (self.M, self.N), float) 
            else:
                self.observation_proba=observation_proba
            if initial_state_proba is None:
                self.initial_state_proba = np.zeros( (self.N, self.N), float ) # pi
            else:
                self.initial_state_proba = initial_state_proba
            # Since everything will be stored in numpy arrays, it is more convenient and compact to 
            # handle words and tags as indices (integer) for a direct access. However, we also need 
            # to keep the mapping between strings (word or tag) and indices. 
            self.make_indexes()
            self.compute_matrix()

        def make_indexes(self):
            """Creates the reverse table that maps states/observations names
            to their index in the probabilities arrays"""
            self.Y_index = {}
            for i in range(self.N):
                self.Y_index[self.omega_Y[i]] = i
            self.X_index = {}
            for i in range(self.M):
                self.X_index[self.omega_X[i]] = i
        
        # Fonction initialisant les matrices de paramètres à l'aide des dictionnaire
        # passés en paramètres
        def compute_matrix(self):
            trans3 = np.zeros( (self.N, self.N, self.N), float)
            for (y1, y2, y3), v in self.transition_proba_3.items():
                trans3[self.Y_index[y1], self.Y_index[y2], self.Y_index[y3]] = v
            self.transition_proba_3 = trans3
            
            trans2 = np.zeros( (self.N, self.N), float)
            for (y1, y2), v in self.transition_proba_2.items():
                trans2[self.Y_index[y1], self.Y_index[y2]] = v
            self.transition_proba_2 = trans2
            
            obs = np.zeros( (self.M, self.N), float)            
            for (x, y), v in self.observation_proba.items():
                obs[self.X_index[x], self.Y_index[y]] = v
            self.observation_proba = obs
            
            init0 = np.zeros( (self.N,), float )
            for y, v in self.initial_state_proba.items():
                init0[self.Y_index[y]] = v
            self.initial_state_proba = init0
        
        # Algorithme de Viterbi au premier ordre
        def viterbi_first_order(self, seq):
            mu = [{}]
            path = {}
            for y in self.omega_Y:
                mu[0][y] = self.initial_state_proba[self.Y_index[y]] * \
                            self.observation_proba[self.X_index[seq[0][0]]][self.Y_index[y]]
                path[y] = [y]

            for t in range(1, len(seq)):
                mu.append({})
                newpath = {}
                indexed_word = seq[t][0]        
                for y in self.omega_Y:
                    tmp = []
                    for y0 in self.omega_Y:
                        a = (mu[t-1][y0] * self.transition_proba_2[self.Y_index[y]][self.Y_index[y0]] * \
                             self.observation_proba[self.X_index[indexed_word]][self.Y_index[y]], y0)
                        tmp.append(a)
                    prob, state = max(tmp)
                    mu[t][y] = prob
                    newpath[y] = path[state] + [y]
                path = newpath

            prob, state = max((mu[len(seq)-1][y], y) for y in self.omega_Y)
            return path[state]
        
        # Algorithme de Viterbi au second ordre
        def viterbi_second_order(self, seq):
                
            d = []
            c = []

            for k in range(len(seq)):
                c.append(np.zeros((len(self.omega_Y), len(self.omega_Y))))
                if k == 0:
                    d.append(np.zeros((len(self.omega_Y))))
                else:
                    d.append(np.zeros((len(self.omega_Y), len(self.omega_Y))))

            for l in range(len(self.omega_Y)):
                d[0][l] = self.initial_state_proba[l] * \
                          self.observation_proba[self.X_index[seq[0][0]]][l]

            if len(seq) > 1:
                for l in range(len(self.omega_Y)):
                    for m in range(len(self.omega_Y)):
                        d[1][l, m] = d[0][l] * self.transition_proba_2[m][l] * \
                                     self.observation_proba[self.X_index[seq[1][0]]][m]

            for i in range(2, len(seq)):
                for m in range(len(self.omega_Y)):
                    for n in range(len(self.omega_Y)):
                        tmp_max_d = np.zeros((len(self.omega_Y)))
                        tmp_argmax_c = np.zeros((len(self.omega_Y)))
                        for l in range(len(self.omega_Y)):
                            d_tmp = d[i-1][l, m] * \
                                    self.transition_proba_3[n][m][l] * \
                                    self.observation_proba[self.X_index[seq[i][0]]][n]
                            c_tmp = d[i-1][l, m] * self.transition_proba_3[n][m][l]
                            tmp_max_d[l] = d_tmp
                            tmp_argmax_c[l] = c_tmp
                        d[i][m, n] = np.max(tmp_max_d)
                        c[i][m, n] = np.argmax(tmp_argmax_c)

            K = len(seq) - 1
            x = [None] * len(seq)

            if len(seq) == 1:
                k_state = np.unravel_index(np.argmax(d[K], axis=None), d[K].shape)
                f_state = k_state[0]
                x[K] = self.omega_Y[f_state]
            else:
                k_kMoinsUn_states = np.unravel_index(np.argmax(d[K], axis=None), d[K].shape)
                k_state_index = k_kMoinsUn_states[1]
                kMoinsUn_state_index = k_kMoinsUn_states[0]
                x[K] = self.omega_Y[k_state_index]
                x[K-1] = self.omega_Y[kMoinsUn_state_index]

            for i in range(K-2, -1, -1):
                index = c[i+2][self.Y_index[x[i+1]], self.Y_index[x[i+2]]]
                index = int(index)
                x[i] = self.omega_Y[index]

            return x
        
        # Fonction évaluant les performance du modèle HMM premier ordre
        def evaluate_fst_order(self, data):
            error = 0
            error_produced = 0
            error_corrected = 0
            total = 0
            for i, seq in enumerate(data):
                res = self.viterbi_first_order(seq)
                for y_pred, (x, y) in list(zip(res, seq)):
                    if x != y and y_pred == y:
                        error_corrected += 1
                    if x == y and y_pred != y:
                        error_produced += 1
                    error += 1 if y_pred != y else 0
                    total += 1
            print("The accuracy of the model(HMM first order) is : ", round((1 - error / total)*100, 2), " %")
            print("The number of error produced is : ", error_produced)
            print("The number of typed error corrected is : ", error_corrected)
        
        # Fonction évaluant les performance du modèle HMM second ordre
        def evaluate_scd_order(self, data):
            error = 0
            error_produced = 0
            error_corrected = 0
            total = 0
            for i, seq in enumerate(data):
                res = self.viterbi_second_order(seq)
                for y_pred, (x, y) in list(zip(res, seq)):
                    if x != y and y_pred == y:
                        error_corrected += 1
                    if x == y and y_pred != y:
                        error_produced += 1
                    error += 1 if y_pred != y else 0
                    total += 1
            print("The accuracy of the model(HMM second order) is : ", round((1 - error / total)*100, 2), " %")
            print("The number of error produced is : ", error_produced)
            print("The number of typed error corrected is : ", error_corrected)

Fonction permettant de comptre le nombre d'occurence de chaque mot, chaque tags et de chaque pairs (mot, tag).

In [4]:
# Fonction permettant de comptre le nombre d'occurence de chaque mot, chaque tags et de chaque
# pairs (mot, tag)
def make_words_tags_count(data):
    count_words = {}
    tags = {}
    pairs = {}
    for sent in data:
        for w,t in sent:
            if w in count_words:
                count_words[w]+=1
            else:
                count_words[w]=1
            if t in tags:
                tags[t]+=1
            else:
                tags[t]=1
            if (w,t) in pairs:
                pairs[(w,t)]+=1
            else:
                pairs[(w,t)]=1
    return count_words, tags, pairs

Fonction permettant de filtrer le vocabulaire, et ainsi de restreindre la taille du vocabulaire au nombre d'occurence souhaité.

In [5]:
# Fonction permettant de filtrer le vocabulaire, et ainsi de restreindre la taille du
# vocabulaire au nombre d'occurence souhaité
def filter_dict(data, limit_occ):
    words,tags,_=make_words_tags_count(data)
    res = []
    for s in data:
        sent = []
        for w,t in s:
            if words[w] > limit_occ:
                sent.append((w,t))
            else:
                sent.append((UNK,t))
        res.append(sent)
    return res

Création des fonctions permettant de calculer la matrice des observations.

In [6]:
# Calcul P(X = mot | Y = tag)
def proba_observation_Y_X(mot, tag, words, pairs):
    if (mot,tag) in pairs:
        return pairs[(mot,tag)] / words[mot]
    else:
        return 0
# Création du dictionnaire servant à initialiser la matrice d'observation
def make_all_proba_observation_Y_X(data):
    words,tags,pairs=make_words_tags_count(data)
    res = {}
    for (w,t),c in pairs.items():
        res[(w,t)]=proba_observation_Y_X(w,t,words,pairs)
    s = sum(v for k, v in res.items())
    res = {k: v / s for k, v in res.items()}
    return res

Création des fonctions permettant le calcul de la matrice de transition du type :
P(Y = a | Y = b).

In [7]:
# Calcul P(Y = tag1 | Y = tag0)
def proba_transition_Y1_Y0(tag1, tag0, pairs, tags):
    if (tag1,tag0) in pairs:
        return pairs[(tag1, tag0)] / tags[tag0]
    else:
        return 0
# Compte le nombre de pairs de tags
def make_tags_pairs(data):
    pairs = {}
    for s in data:
        for i in range(len(s) - 1):
            if (s[i+1][1], s[i][1]) in pairs:
                pairs[s[i+1][1], s[i][1]]+=1
            else:
                pairs[s[i+1][1], s[i][1]]=1
    return pairs

# Création du dictionnaire servant à initialiser la matrice de transition à l'ordre 1
def make_all_proba_transition_Y1_Y0(data):
    _,tags,_=make_words_tags_count(data)
    pairs=make_tags_pairs(data)
    res = {}
    for (y1,y0),c in pairs.items():
        res[(y1,y0)]=proba_transition_Y1_Y0(y1,y0, pairs, tags)
    s = sum(v for k, v in res.items())
    res = {k: v / s for k, v in res.items()}
    return res

Création de toutes les fonctions permettant l'initialisation de la matrice de transition.
Ici, il s'agit de la matrice de transition construite de la manière suivante:

P(Yi = a | Yi-1 = b, Yi-2 = c) = transition.proba[a][b][c]

In [8]:
# Calcul P(Y = tag2 | Y = tag1, Y = tag0)
def proba_transition_Y2_Y1_Y0(tag2, tag1, tag0, triplets, pairs):
    if (tag2,tag1,tag0) in triplets:
        return triplets[(tag2, tag1, tag0)] / pairs[(tag1, tag0)]
    else:
        return 0
# Compte le nombre de triplets de tags
def make_tags_triplets(data):
    triplets = {}
    for s in data:
        for i in range(len(s) - 2):
            if (s[i+2][1], s[i+1][1], s[i][1]) in triplets:
                triplets[s[i+2][1], s[i+1][1], s[i][1]]+=1
            else:
                triplets[s[i+2][1], s[i+1][1], s[i][1]]=1
    return triplets

# Création du dictionnaire servant à initialiser la matrice de transition à l'ordre 2
def make_all_proba_transition_Y2_Y1_Y0(data):
    triplets=make_tags_triplets(data)
    pairs=make_tags_pairs(data)
    res = {}
    for (y2,y1,y0),c in triplets.items():
        res[(y2,y1,y0)]=proba_transition_Y2_Y1_Y0(y2,y1,y0, triplets, pairs)
    s = sum(v for k, v in res.items())
    res = {k: v / s for k, v in res.items()}
    return res

Création des probabilités de tags initiales

In [9]:
# Création du dictionnaire servant à la l'initialisation de la matrice "Pi" : "initial_state_proba"
def makePi0(data):
    res = {}
    total = 0
    for s in data:
        if s[0][1] in res:
            res[s[0][1]] += 1
        else:
            res[s[0][1]] = 1
        total += 1
    for k,v in res.items():
        res[k] /= total
    return res

Fonction d'initialisation du HMM

In [10]:
# Initialisation du modèle HMM avec toutes les matrices de paramètres correspondantes.
def initHMM(data):
    count_words, count_tags, pairs = make_words_tags_count(data)
    obs = make_all_proba_observation_Y_X(data)
    trans_2 = make_all_proba_transition_Y1_Y0(data)
    trans_3 = make_all_proba_transition_Y2_Y1_Y0(data)
    return HMM([t for t,_ in count_tags.items()], [w for w,_ in count_words.items()], trans_2, trans_3, obs, makePi0(data))

### Chargement des données

In [11]:
data_train10 = pickle.load( open( "../typos-data/train10.pkl", "rb" ))
data_test10 = pickle.load(open("../typos-data/test10.pkl", "rb"))

data_train20 = pickle.load(open("../typos-data/train20.pkl", "rb"))
data_test20 = pickle.load(open("../typos-data/test20.pkl", "rb"))

In [13]:
# Process des données pour le modèle naïf
data_train10naïf = process_data(data_train10, 10)
data_train20naïf = process_data(data_train20, 10)

# Process des données pour les HMM
data_train10hmm = filter_dict(data_train10,10)
data_train20hmm = filter_dict(data_train20, 10)

print("Nombre de sequences d'entrainement dans train10.pkl : ", len(data_train10))
print("Nombre de sequences d'entrainement dans train20.pkl : ", len(data_train20), "\n")
print("Nombre de sequences de test dans test10.pkl : ", len(data_test10))
print("Nombre de sequences de test dans train20.pkl : ", len(data_test10))

Nombre de sequences d'entrainement dans train10.pkl :  29057
Nombre de sequences d'entrainement dans train20.pkl :  27184 

Nombre de sequences de test dans test10.pkl :  1501
Nombre de sequences de test dans train20.pkl :  1501


### Test du modèle naïf avec les données (train10.pkl & test10.pkl)

In [14]:
tags, l_tags, count_words, count_of_pairs = compute_params(data_train10naïf)

In [15]:
accuracy = eval_model(data_test10, data_train10naïf, count_of_pairs, count_words, l_tags)
print("Pourcentage de précision pour modèle naïf sur les données 10% erronnées : ", accuracy, " %")

Pourcentage de précision pour modèle naïf sur les données 10% erronnées :  90.0  %


In [16]:
tags, l_tags, count_words, count_of_pairs = compute_params(data_train20naïf)

In [17]:
accuracy = eval_model(data_test20, data_train20naïf, count_of_pairs, count_words, l_tags)
print("Pourcentage de précision pour modèle naïf sur les données 20% erronnées : ", accuracy, " %")

Pourcentage de précision pour modèle naïf sur les données 20% erronnées :  81.9  %


### Test du HMM avec les données (train10.pkl & test10.pkl)

In [18]:
print("Initialisation du modèle HMM : \n")
HMM10 = initHMM(data_train10hmm)

Initialisation du modèle HMM : 

HMM created with: 
26 states
26 observations


In [19]:
HMM10.evaluate_fst_order(data_test10)

The accuracy of the model(HMM first order) is :  91.5  %
The number of error produced is :  210
The number of typed error corrected is :  333


In [20]:
HMM10.evaluate_scd_order(data_test10)

The accuracy of the model(HMM second order) is :  94.88  %
The number of error produced is :  129
The number of typed error corrected is :  499


### Test du HMM avec les données (train20.pkl & test20.pkl)

In [21]:
print("Initialisation du modèle HMM : \n")
HMM20 = initHMM(data_train20hmm)

Initialisation du modèle HMM : 

HMM created with: 
26 states
26 observations


In [22]:
HMM20.evaluate_fst_order(data_test20)

The accuracy of the model(HMM first order) is :  85.04  %
The number of error produced is :  760
The number of typed error corrected is :  1502


In [23]:
HMM20.evaluate_scd_order(data_test20)

The accuracy of the model(HMM second order) is :  91.06  %
The number of error produced is :  477
The number of typed error corrected is :  2224


## Récapitulatif

Que ce soit sur les données avec 10% ou 20% d'erreurs le HMM au second ordre et toujours plus performant que celui au premier ordre (ex : 94,88% contre 91,5%). Evidemment le modèle naïf implémenté pour le tp1 à été repris et testé sur ces données également. Les performances de celui-ci sont en dessous des deux modèles HMM (90% au mieux pour le modèles naïf).
On se rend compte également que les HMM ainsi testé parviennent à corriger pas mal d'erreurs et n'en produisent pas tant que ça. Dans tous les cas des HMM implémentés ci-dessus, ceux-ci corrigent plus d'erreurs qu'il n'en produisent.